In [50]:
import pandas as pd
column_names = ['Code', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
min_temps = pd.read_csv('../datasets/raw_data/min_temp.csv', delimiter='\s+', names=column_names)
max_temps = pd.read_csv('../datasets/raw_data/max_temp.csv', delimiter='\s+', names=column_names)
avg_temps = pd.read_csv('../datasets/raw_data/avg_temp.csv', delimiter='\s+', names=column_names)

# Fill the code
min_temps["Code"] = min_temps["Code"].astype(str).str.zfill(10)
max_temps["Code"] = max_temps["Code"].astype(str).str.zfill(10)
avg_temps["Code"] = avg_temps["Code"].astype(str).str.zfill(10)

# Add the type of measure
min_temps["Type"] = "Minimum"
max_temps["Type"] = "Maximum"
avg_temps["Type"] = "Average"

min_temps.head(5)

,Code,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Type
0,0010281895,33.4,26.8,42.4,51.2,58.4,66.5,68.8,69.3,66.4,45.2,41.4,32.7,Minimum
1,0010281896,34.0,36.4,41.2,55.6,63.9,66.9,70.7,70.4,62.6,50.6,45.8,35.1,Minimum
2,0010281897,31.4,41.2,50.4,50.1,55.9,68.3,70.2,68.1,62.5,53.1,41.4,36.9,Minimum
3,0010281898,38.8,34.2,48.5,46.4,60.2,68.0,69.6,69.2,65.1,49.6,38.7,32.5,Minimum
4,0010281899,33.1,28.6,43.5,50.8,63.6,67.6,68.9,69.9,59.4,54.0,42.8,33.8,Minimum


In [51]:
regions = pd.read_csv('../datasets/raw_data/regions_id.csv')
regions["ID"] = regions["ID"].astype(str)
regions["ID"] = regions["ID"].str.zfill(3)
regions.head(5)

,ID,Region
0,001,Alabama
1,002,Arizona
2,003,Arkansas
3,004,California
4,005,Colorado


In [52]:
# Extract the year from the code
min_temps["Year"] = min_temps["Code"].astype(str).str[-4:]
min_temps["Year"] = min_temps["Year"].astype(int)
max_temps["Year"] = max_temps["Code"].astype(str).str[-4:]
max_temps["Year"] = max_temps["Year"].astype(int)
avg_temps["Year"] = avg_temps["Code"].astype(str).str[-4:]
avg_temps["Year"] = avg_temps["Year"].astype(int)

# Extract the region
min_temps["RegionID"] = min_temps["Code"].astype(str).str[:3]
min_temps["Year"] = min_temps["Year"].astype(int)
max_temps["RegionID"] = max_temps["Code"].astype(str).str[:3]
max_temps["Year"] = max_temps["Year"].astype(int)
avg_temps["RegionID"] = avg_temps["Code"].astype(str).str[:3]
avg_temps["Year"] = avg_temps["Year"].astype(int)

# Convert Fahrenheit to Celsius
min_temps[months] = ((min_temps[months] - 32) * 5/9).round(3)
max_temps[months] = ((max_temps[months] - 32) * 5/9).round(3)
avg_temps[months] = ((avg_temps[months] - 32) * 5/9).round(3)


# Merge DataFrames based on 'RegionID' and 'ID'
min_temps = pd.merge(min_temps, regions, left_on='RegionID', right_on='ID', how='left')
max_temps = pd.merge(max_temps, regions, left_on='RegionID', right_on='ID', how='left')
avg_temps = pd.merge(avg_temps, regions, left_on='RegionID', right_on='ID', how='left')


# Drop the redundant 'ID' column and rename 'Region' to 'RegionName'
min_temps = min_temps.drop(columns=['ID']).rename(columns={'Region': 'RegionName'})
max_temps = max_temps.drop(columns=['ID']).rename(columns={'Region': 'RegionName'})
avg_temps = avg_temps.drop(columns=['ID']).rename(columns={'Region': 'RegionName'})


# To set the new order of the columns
new_columns = ["Code", "Type", "RegionID", "RegionName", "Year"] + months
min_temps = min_temps[new_columns]
max_temps = max_temps[new_columns]
avg_temps = avg_temps[new_columns]


min_temps.head(5)

,Code,Type,RegionID,RegionName,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,0010281895,Minimum,001,Alabama,1895,0.778,-2.889,5.778,10.667,14.667,19.167,20.444,20.722,19.111,7.333,5.222,0.389
1,0010281896,Minimum,001,Alabama,1896,1.111,2.444,5.111,13.111,17.722,19.389,21.500,21.333,17.000,10.333,7.667,1.722
2,0010281897,Minimum,001,Alabama,1897,-0.333,5.111,10.222,10.056,13.278,20.167,21.222,20.056,16.944,11.722,5.222,2.722
3,0010281898,Minimum,001,Alabama,1898,3.778,1.222,9.167,8.000,15.667,20.000,20.889,20.667,18.389,9.778,3.722,0.278
4,0010281899,Minimum,001,Alabama,1899,0.611,-1.889,6.389,10.444,17.556,19.778,20.500,21.056,15.222,12.222,6.000,1.000


In [53]:
# Drop null regions
min_temps.dropna(subset=['RegionName'], inplace=True)
max_temps.dropna(subset=['RegionName'], inplace=True)
avg_temps.dropna(subset=['RegionName'], inplace=True)

min_temps['RegionName'] = min_temps['RegionName'].replace('National (contiguous 48 States)', 'National')
max_temps['RegionName'] = max_temps['RegionName'].replace('National (contiguous 48 States)', 'National')
avg_temps['RegionName'] = avg_temps['RegionName'].replace('National (contiguous 48 States)', 'National')

In [54]:
# Union of dataframes
temps = pd.concat([min_temps, max_temps, avg_temps])
temps.head(10)

,Code,Type,RegionID,RegionName,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,0010281895,Minimum,001,Alabama,1895,0.778,-2.889,5.778,10.667,14.667,19.167,20.444,20.722,19.111,7.333,5.222,0.389
1,0010281896,Minimum,001,Alabama,1896,1.111,2.444,5.111,13.111,17.722,19.389,21.500,21.333,17.000,10.333,7.667,1.722
2,0010281897,Minimum,001,Alabama,1897,-0.333,5.111,10.222,10.056,13.278,20.167,21.222,20.056,16.944,11.722,5.222,2.722
3,0010281898,Minimum,001,Alabama,1898,3.778,1.222,9.167,8.000,15.667,20.000,20.889,20.667,18.389,9.778,3.722,0.278
4,0010281899,Minimum,001,Alabama,1899,0.611,-1.889,6.389,10.444,17.556,19.778,20.500,21.056,15.222,12.222,6.000,1.000
5,0010281900,Minimum,001,Alabama,1900,-0.056,0.222,5.167,10.889,14.667,19.667,20.611,20.500,18.889,14.556,6.111,2.056
6,0010281901,Minimum,001,Alabama,1901,1.389,0.167,5.056,7.889,13.944,19.167,21.000,19.889,16.111,9.333,1.500,-0.833
7,0010281902,Minimum,001,Alabama,1902,-0.611,-0.778,6.667,9.611,17.278,19.889,21.167,21.111,16.944,10.333,7.722,1.611
8,0010281903,Minimum,001,Alabama,1903,0.722,2.667,10.278,9.000,14.833,16.778,20.444,20.778,15.444,9.667,3.722,-2.611
9,0010281904,Minimum,001,Alabama,1904,-1.278,3.111,7.667,8.500,13.444,18.278,19.222,19.778,17.611,9.556,4.000,1.556


In [55]:
# Save into CSV files
min_temps.to_csv("../datasets/min_temps.csv", index=False)
max_temps.to_csv("../datasets/max_temps.csv", index=False)
avg_temps.to_csv("../datasets/avg_temps.csv", index=False)
temps.to_csv("../datasets/temps.csv", index=False)